In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time
import re

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.hani.co.kr/arti?page={}"
article_base = "https://www.hani.co.kr/"

dict = defaultdict(list)
for page in range(1, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출
    
    ul_elem = driver.find_element(By.CSS_SELECTOR, 'ul')
    li_list = ul_elem.find_elements(By.TAG_NAME, 'li')
    print(li_list)
# 기사 정보 출력
    for li in li_list: 
               # 링크 태그 찾기
        a_tag = li.find_element(By.CSS_SELECTOR, 'a')
        href = a_tag.get_attribute('href')
        article_url = href
        print(article_url)

        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        dict['Category'].append(article_soup.select_one('#renewal2023 > div.ArticleDetailView_breadcrumb___UwRC > a:nth-child(1)').get_text(strip=True))
        dict['Title'].append(article_soup.select_one('#renewal2023 > h3').get_text(strip=True))
        

        date_element = article_soup.select_one('#renewal2023 ul.ArticleDetailView_dateList__tniXJ li span')
        if date_element:
            date_text = date_element.get_text(strip=True).split()[0]
            dict['Date'].append(date_text)
        else:
            dict['Date'].append('날짜 없음')

        
        sentences = []

        paragraphs = article_soup.select_one('#renewal2023 > div.article-text > p')
        for p in paragraphs:
            text = p.get_text(strip=True)
            if text:  # 빈 텍스트는 제외
                sentences.append(text)

    
        dict['Contents'].append(' '.join(sentences))

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)


[<selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db489d", element="f.307AB9F72A2A4AD4261EBD5631593A98.d.603AE9D136276E20DE547D4D266D0A27.e.15")>, <selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db489d", element="f.307AB9F72A2A4AD4261EBD5631593A98.d.603AE9D136276E20DE547D4D266D0A27.e.16")>, <selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db489d", element="f.307AB9F72A2A4AD4261EBD5631593A98.d.603AE9D136276E20DE547D4D266D0A27.e.17")>, <selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db489d", element="f.307AB9F72A2A4AD4261EBD5631593A98.d.603AE9D136276E20DE547D4D266D0A27.e.18")>, <selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db489d", element="f.307AB9F72A2A4AD4261EBD5631593A98.d.603AE9D136276E20DE547D4D266D0A27.e.19")>, <selenium.webdriver.remote.webelement.WebElement (session="5f70396e3f56476ebba9355579db48

In [3]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_han.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)